# Predictive Analysis of Violent Sexual Crimes
**Emily Nguyen, Kaylynn Francisco-Nelson, Angela Iraya**

## Data Description & Cleaning
---

In [ ]:
# imports/setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv("../data/sexual_crimes_cleaned.csv") # pre-cleaned data from phase 1
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Time_Formatted,Hour,Time_Category,Date_Rptd,DATE_OCC,Year,Month,DayOfWeek,Reporting_Delay,Delay_Category
0,202013579,08/18/2020 12:00:00 AM,08/13/2020 12:00:00 AM,100,20,Olympic,2014,2,860,BATTERY WITH SEXUAL CONTACT,...,1:00,1,Night (0-6),2020-08-18,2020-08-13,2020,Aug,Thu,5,1-7 days
1,211907205,04/01/2021 12:00:00 AM,02/22/2020 12:00:00 AM,1,19,Mission,1915,1,122,"RAPE, ATTEMPTED",...,0:01,0,Night (0-6),2021-04-01,2020-02-22,2020,Feb,Sat,404,Over 1 year
2,221908151,04/12/2022 12:00:00 AM,10/01/2020 12:00:00 AM,1,19,Mission,1988,1,821,SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO...,...,0:01,0,Night (0-6),2022-04-12,2020-10-01,2020,Oct,Thu,558,Over 1 year
3,201904093,01/02/2020 12:00:00 AM,01/02/2020 12:00:00 AM,2025,19,Mission,1901,1,121,"RAPE, FORCIBLE",...,20:25,20,Evening (18-24),2020-01-02,2020-01-02,2020,Jan,Thu,0,0-1 days
4,210708216,04/29/2021 12:00:00 AM,05/01/2020 12:00:00 AM,1330,7,Wilshire,785,1,121,"RAPE, FORCIBLE",...,13:30,13,Afternoon (12-18),2021-04-29,2020-05-01,2020,May,Fri,363,1-12 months


In [ ]:
# printing column names - including transformed cols from phase 1
df.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3',
       'Crm Cd 4', 'Vict Descent Full', 'Vict Age Group', 'Time_Formatted',
       'Hour', 'Time_Category', 'Date_Rptd', 'DATE_OCC', 'Year', 'Month',
       'DayOfWeek', 'Reporting_Delay', 'Delay_Category'],
      dtype='object')

In [24]:
# get the number of rows and columns
num_rows, num_cols = df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 12371
Number of columns: 34


In [25]:
crime_counts = (
    df
    .groupby(['Crm Cd', 'Crm Cd Desc'])
    .size()
    .reset_index(name='Case_Count')
    .sort_values(by='Case_Count', ascending=False)
)

print(crime_counts)

    Crm Cd                                        Crm Cd Desc  Case_Count
10     860                        BATTERY WITH SEXUAL CONTACT        4134
0      121                                     RAPE, FORCIBLE        3739
4      815                SEXUAL PENETRATION W/FOREIGN OBJECT        1309
3      810  SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W...        1072
5      820                                    ORAL COPULATION         718
6      821  SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO...         527
7      822            HUMAN TRAFFICKING - COMMERCIAL SEX ACTS         456
1      122                                    RAPE, ATTEMPTED         318
2      760                    LEWD/LASCIVIOUS ACTS WITH CHILD          85
9      840  BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT...           7
8      830       INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)           6


## Representation Comparison: Baseline vs Improved
---
We compare model behavior under 2 feature representations:
- **Baseline**: Closest to Phase 1
- **Improved**: Tranformations from our Feature Representation Audit

We trained 3 models (Decision Tree, Random Forest, XGBoost), considering: 
- Accuracy, Precision, Recall, F1 (macro)
- Fiarness metrics
- Intepretability: permutation importance + SHAP for tree models


In [27]:
# imports
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score, classification_report, accuracy_score, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

### Baseline Models
In phase 1, we transformed some columns used in our model, including Delay_Cateogry, DayOfWeek, Vict Descent Full, Vict Age Group, and Time_Category. We used numerical features that were left untransformed, while the categorical features were one-hot encoded. 

We split the dataset 80/20 while preserving class distribution (stratify=y) to ensure each delay category is represented in both training and test sets.

In [40]:
# target feature
target = "Delay_Category"

# features that we want to use for prediction
numeric_features = [
    "Vict Age", "TIME OCC", "Hour", "Year", "Month", 
    "Part 1-2", "Rpt Dist No" # new features we want to explore
    ]
categorical_features = [
    "DayOfWeek", "Vict Sex", "Vict Descent", "Vict Age Group", 
    "AREA NAME", "Premis Desc", "Crm Cd Desc", 
    "Weapon Desc", "Weapon Used Cd",
    # new features we want to explore (including categories made in phase 1)
    "Time_Category", "DayOfWeek", "DATE OCC"
]

X = df[numeric_features + categorical_features]
y = df[target].astype(str)

**Train/Test Split**

In [41]:
# train/test split before encoding
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape, " Test size:", X_test.shape)

Train size: (9896, 19)  Test size: (2475, 19)


**Preprocessing**

In [42]:
# preprocessing: one-hot encode categorical, passthrough numerics
ohe = OneHotEncoder(handle_unknown="ignore")
preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", ohe, categorical_features),
    ]
)

In [ ]:
#   - Builds a pipeline: preprocessing (preprocess) + classifier (estimator)
#   - Runs GridSearchCV with the given parameter grid and 5-fold CV, scoring by F1-macro
#   - Fits the tuned model on the training data
#   - Predicts labels on the test set
#   - Returns a dictionary with: model name, best hyperparameters found, best cross-validation F1 score, test accuracy, test F1-macro, the trained best estimator itself
def tune_and_eval(model_name, estimator, param_grid, X_train, y_train, X_test, y_test):
    pipe = Pipeline([("prep", preprocess), ("clf", estimator)])
    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        scoring="f1_macro",
        cv=5,
        n_jobs=-1,
        verbose=0
    )
    grid.fit(X_train, y_train)
    best = grid.best_estimator_
    y_pred = best.predict(X_test)
    return {
        "model": model_name,
        "best_params": grid.best_params_
        ,
        "cv_f1_macro": grid.best_score_,
        "test_accuracy": accuracy_score(y_test, y_pred),
        "test_f1_macro": f1_score(y_test, y_pred, average="macro"),
        "estimator": best
    }

results = []

# Decision Tree
results.append(tune_and_eval(
    "DecisionTree",
    DecisionTreeClassifier(random_state=42),
    {
        "clf__criterion": ["gini", "log_loss"],
        "clf__max_depth": [5, 10, 20, None],
        "clf__min_samples_leaf": [1, 5, 10, 50],
        "clf__max_features": [None, "sqrt", "log2"],
        "clf__class_weight": [None, "balanced"],
    },
    X_train, y_train, X_test, y_test
))

# Random Forest
results.append(tune_and_eval(
    "RandomForest",
    RandomForestClassifier(random_state=42, n_jobs=-1),
    {
        "clf__n_estimators": [300],
        "clf__max_depth": [None, 8, 16],
        "clf__min_samples_leaf": [1, 10, 50],
        "clf__class_weight": [None, "balanced_subsample"],
    },
    X_train, y_train, X_test, y_test
))

# XGBoost
results.append(tune_and_eval(
    "XGBoost",
    XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    {
        "clf__n_estimators": [100, 200, 300],
        "clf__max_depth": [4, 6, 8],
        "clf__learning_rate": [0.01, 0.1, 0.3],
        "clf__subsample": [0.8, 1.0],
        "clf__colsample_bytree": [0.8, 1.0],
        
        # use this instead if XGBoost is performing quickly enough to add more params
        # "clf__n_estimators": [200, 400],
        # "clf__max_depth": [4, 6, 8],
        # "clf__learning_rate": [0.01, 0.05, 0.1],
        # "clf__subsample": [0.7, 0.8, 1.0],
        # "clf__colsample_bytree": [0.7, 0.8, 1.0],
        # "clf__gamma": [0, 1, 5],
        # "clf__reg_lambda": [1, 5, 10],
        # "clf__reg_alpha": [0, 0.1, 1],
    },
    X_train, y_train, X_test, y_test
))

ValueError: 
All the 240 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py", line 1131, in _hstack
    check_array(X, accept_sparse=True, ensure_all_finite=False)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py", line 1053, in check_array
    array = _asarray_with_order(array, order=order, dtype=dtype, xp=xp)
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/_array_api.py", line 757, in _asarray_with_order


### Transformations
**Previously Transformed Columns from Phase 1 include:**
- Vict Descent Full
- Vict Age Group
- Time_Formatted
- Hour
- Time_Category
- Year
- Month
- DayOfWeek
- Reporting_Delay
- Delay_Category

**Transforming "Premis Desc"**
- "Premis_Category": Grouped bins by type of premises (e.g., "Residential", "Commerical/Business", etc.)

In [44]:
# Transform "Premis Desc" into high-level categories
desc_upper = df["Premis Desc"].str.upper()

conditions = [
    # Residential
    desc_upper.str.contains(r'SINGLE FAMILY DWELLING|MULTI-UNIT DWELLING|CONDOMINIUM|TOWNHOUSE|MOBILE HOME|OTHER RESIDENCE|FRAT HOUSE|DORMITORY|SRO|GROUP HOME|HOTEL|MOTEL|TRANSITIONAL HOUSING|FOSTER HOME|VACATION RENTAL', na=False),

    # Vehicle/Transportation
    desc_upper.str.contains(r'VEHICLE|AUTO|CAR|TRUCK|BUS|MTA|METROLINK|GREYHOUND|PARKING LOT|PARKING GARAGE|DRIVEWAY|GARAGE|CARPORT|TOW YARD|BUS STOP|TRAIN DEPOT|TERMINAL|STATION', na=False),

    # Public Outdoor
    desc_upper.str.contains(r'STREET|SIDEWALK|HIGHWAY|ROAD|ALLEY|FREEWAY|UNDERPASS|PARK|PLAYGROUND|BEACH|PIER|RIVER BED|VACANT LOT', na=False),

    # Commercial/Business
    desc_upper.str.contains(r'RESTAURANT|FAST FOOD|COFFEE SHOP|BAR|NIGHT CLUB|TAVERN|STORE|MARKET|MALL|SHOPPING|RETAIL|DEPARTMENT|DRUG STORE|LIQUOR|OFFICE|BUSINESS|BANK|PAWN SHOP|SWAP MEET|GAS STATION|MINI-MART|AUTO REPAIR|CAR WASH', na=False),

    # Institutional
    desc_upper.str.contains(r'SCHOOL|COLLEGE|UNIVERSITY|PRESCHOOL|DAY CARE|HOSPITAL|MEDICAL|CLINIC|NURSING|HOSPICE|METHADONE|CHURCH|SYNAGOGUE|TEMPLE|LIBRARY|MUSEUM|GOVERNMENT|POLICE|DETENTION|JAIL', na=False),

    # Entertainment/Recreation
    desc_upper.str.contains(r'ENTERTAINMENT|COMEDY CLUB|THEATRE|MOVIE|BOWLING|ARCADE|SPORTS|ARENA|STADIUM|GYM|SPA|POOL|SKATEBOARD|SEX ORIENTED|STRIP CLUB|MASSAGE PARLOR', na=False),

    # Industrial/Construction
    desc_upper.str.contains(r'CONSTRUCTION|FACTORY|WAREHOUSE|MANUFACTURING', na=False),

    # Other/Unknown
    desc_upper.str.contains(r'UNKNOWN|OTHER PREMISE|OTHER BUSINESS|OTHER/OUTSIDE', na=False)
]

choices = [
    "Residential",
    "Vehicle/Transportation",
    "Public Outdoor",
    "Commercial/Business",
    "Institutional",
    "Entertainment/Recreation",
    "Industrial/Construction",
    "Other/Unknown"
]

df["Premis_Category"] = np.select(conditions, choices, default="Other/Unknown")

print(df["Premis_Category"].value_counts())

Premis_Category
Residential                 6377
Public Outdoor              2835
Vehicle/Transportation      1365
Institutional                683
Other/Unknown                524
Commercial/Business          447
Entertainment/Recreation     116
Industrial/Construction       24
Name: count, dtype: int64


**Transforming "Weapon Desc"**
- "Weapon_Category": Consolidated individual weapon descriptions into higher-level categories to simplify interpretation and distinguish between weapon use vs. no weapon use.

In [45]:
# looking at unique values in "Weapon Desc"
df['Weapon Desc'].unique()

array(['STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)', nan,
       'UNKNOWN WEAPON/OTHER WEAPON', 'HAND GUN', 'PHYSICAL PRESENCE',
       'OTHER KNIFE', 'VERBAL THREAT', 'KNIFE WITH BLADE 6INCHES OR LESS',
       'CLUB/BAT', 'BOTTLE', 'UNKNOWN FIREARM', 'SEMI-AUTOMATIC PISTOL',
       'OTHER FIREARM', 'STICK', 'FOLDING KNIFE',
       'KNIFE WITH BLADE OVER 6 INCHES IN LENGTH', 'KITCHEN KNIFE',
       'ROPE/LIGATURE', 'AUTOMATIC WEAPON/SUB-MACHINE GUN',
       'BLUNT INSTRUMENT', 'TIRE IRON', 'SIMULATED GUN',
       'PIPE/METAL PIPE', 'FIXED OBJECT', 'SEMI-AUTOMATIC RIFLE',
       'HAMMER', 'MACE/PEPPER SPRAY', 'AXE', 'REVOLVER',
       'CAUSTIC CHEMICAL/POISON', 'SCISSORS', 'OTHER CUTTING INSTRUMENT',
       'STUN GUN', 'LIQUOR/DRUGS', 'TOY GUN',
       'HECKLER & KOCH 93 SEMIAUTOMATIC ASSAULT RIFLE', 'BOW AND ARROW',
       'MACHETE', 'DIRK/DAGGER', 'RAZOR BLADE', 'SCALDING LIQUID',
       'HECKLER & KOCH 91 SEMIAUTOMATIC ASSAULT RIFLE',
       'ROCK/THROWN OBJECT', 'BELT FLAILING I

In [48]:
# Create uppercase version to make matching easier
desc_upper = df["Weapon Desc"].str.upper()

# Define boolean masks for each category
conditions = [
    # No Weapon / Physical Force
    desc_upper.isna() | desc_upper.str.contains(r'STRONG-ARM|PHYSICAL PRESENCE|VERBAL THREAT', na=False),
    
    # Firearm
    desc_upper.str.contains(r'HAND GUN|REVOLVER|SEMI-AUTOMATIC|AUTOMATIC|OTHER FIREARM|UNKNOWN FIREARM|SIMULATED GUN|TOY GUN|HECKLER|RIFLE', na=False),
    
    # Knife / Sharp Object
    desc_upper.str.contains(r'KNIFE|DIRK|DAGGER|MACHETE|RAZOR|SCISSORS|CUTTING', na=False),
    
    # Blunt Object
    desc_upper.str.contains(r'CLUB|BAT|STICK|PIPE|TIRE IRON|HAMMER|ROCK|BELT|CHAIN|BLUNT|FIXED OBJECT|BOTTLE', na=False),
    
    # Chemical / Explosive / Other
    desc_upper.str.contains(r'MACE|PEPPER SPRAY|CAUSTIC|POISON|SCALDING|LIQUOR|DRUGS|BOW|ARROW|OTHER WEAPON|ROPE|LIGATURE|AXE|STUN GUN', na=False)
]

# Define labels corresponding to the above conditions
choices = [
    "No Weapon, Physical Force",
    "Firearm",
    "Knife, Sharp Object",
    "Blunt Object",
    "Chemical, Explosive, Other"
]

# Apply transformation
df["Weapon_Category"] = np.select(conditions, choices, default="Other/Unknown")

# Verify transformation
print(df["Weapon_Category"].value_counts())


Weapon_Category
No Weapon, Physical Force     11675
Chemical, Explosive, Other      427
Firearm                         134
Knife, Sharp Object             106
Blunt Object                     29
Name: count, dtype: int64


### Improved Models

In [50]:
# target feature
target = "Delay_Category"

# features that we want to use for prediction
numeric_features = [
    "Hour", "Year", "Month", 
    "Part 1-2", "Rpt Dist No"
]
categorical_features = [
    "Vict Sex", "Vict Descent Full", "Vict Age Group", 
    "AREA NAME", "Premis_Category",
    "Crm Cd Desc", "Weapon_Category",
    "DayOfWeek", "Time_Category", 
]

X = df[numeric_features + categorical_features]
y = df[target].astype(str)

**Train/Test Split**

In [51]:
# train/test split before encoding
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape, " Test size:", X_test.shape)


Train size: (9896, 14)  Test size: (2475, 14)


**Preprocessing**

In [52]:
# preprocessing: one-hot encode categorical, passthrough numerics
ohe = OneHotEncoder(handle_unknown="ignore")
preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", ohe, categorical_features),
    ]
)

In [ ]:
improved_results = []

# Decision Tree
improved_results.append(tune_and_eval(
    "DecisionTree",
    DecisionTreeClassifier(random_state=42),
    {
        "clf__criterion": ["gini", "log_loss"],
        "clf__max_depth": [5, 10, 20, None],
        "clf__min_samples_leaf": [1, 5, 10, 50],
        "clf__max_features": [None, "sqrt", "log2"],
        "clf__class_weight": [None, "balanced"],
    },
    X_train, y_train, X_test, y_test
))

# Random Forest
improved_results.append(tune_and_eval(
    "RandomForest",
    RandomForestClassifier(random_state=42, n_jobs=-1),
    {
        "clf__n_estimators": [300],
        "clf__max_depth": [None, 8, 16],
        "clf__min_samples_leaf": [1, 10, 50],
        "clf__class_weight": [None, "balanced_subsample"],
    },
    X_train, y_train, X_test, y_test
))

# XGBoost
improved_results.append(tune_and_eval(
    "XGBoost",
    XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    {
        "clf__n_estimators": [100, 200, 300],
        "clf__max_depth": [4, 6, 8],
        "clf__learning_rate": [0.01, 0.1, 0.3],
        "clf__subsample": [0.8, 1.0],
        "clf__colsample_bytree": [0.8, 1.0],
        
        # use this instead if XGBoost is performing quickly enough to add more params
        # "clf__n_estimators": [200, 400],
        # "clf__max_depth": [4, 6, 8],
        # "clf__learning_rate": [0.01, 0.05, 0.1],
        # "clf__subsample": [0.7, 0.8, 1.0],
        # "clf__colsample_bytree": [0.7, 0.8, 1.0],
        # "clf__gamma": [0, 1, 5],
        # "clf__reg_lambda": [1, 5, 10],
        # "clf__reg_alpha": [0, 0.1, 1],
    },
    X_train, y_train, X_test, y_test
))